In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
from dataclasses import dataclass, field, fields
import pack_cuda
import pack_vis
import pack_cost
import copy
import time
import pack_ga
from IPython.display import HTML, display, clear_output
pack_cuda.USE_FLOAT32 = True
pack_cuda._ensure_initialized()


local
stop final relax at some point
init CUDA
Detected GPU compute capability: 8.9 (arch=sm_89)
GPU max threads per block: 1024
=== Compiling kernel variant: crystal ===
Defines: ENABLE_CRYSTAL_AXES, ENABLE_OVERLAP_AREA, ENABLE_SEPARATION
Command: /usr/local/cuda/bin/nvcc -O3 -use_fast_math --extra-device-vectorization --ptxas-options=-v,--warn-on-spills -arch=sm_89 -DENABLE_CRYSTAL_AXES -DENABLE_OVERLAP_AREA -DENABLE_SEPARATION -cubin /mnt/d//packing/temp/pack_cuda_saved.cu -o /mnt/d//packing/temp/pack_cuda_crystal.cubin
ptxas info    : 0 bytes gmem, 1172 bytes cmem[3]
ptxas info    : Compiling entry function 'multi_boundary_distance_list_total' for 'sm_89'
ptxas info    : Function properties for multi_boundary_distance_list_total
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 36 registers, 404 bytes cmem[0], 16 bytes cmem[2]
ptxas info    : Compiling entry function 'multi_boundary_list_total' for 'sm_89'
ptxas info    : Function properties fo

In [16]:
import pack_runner
fastMode = False
ga = pack_ga.GA()

ga.N_trees_to_do = np.array([40])
ga.n_generations = 10
ga.population_size = 100
ga.selection_size = [1,2,3,4,5,10,20,30,50]
ga.do_legalize = False
ga.initializer.base_solution.use_fixed_h = True
ga.initializer.fixed_h = cp.array([4.,0.,0.], dtype=kgs.dtype_cp)
ga.reduce_h_threshold = 0.01
ga.reduce_h_amount = 0.03

kgs.profiling=False
ga.run()
print(ga.populations[-1].fitness)
print(ga.best_cost_per_generation)

Generation 0, Trees 40, Best cost: 0.05277536, Est: 0.13193841, h: 4.000000
Generation 1, Trees 40, Best cost: 0.01487751, Est: 0.03719376, h: 4.000000
Generation 2, Trees 40, Best cost: 0.00756818, Est: 0.01892045, h: 4.000000
Generation 3, Trees 40, Best cost: 0.00527892, Est: 0.01319729, h: 3.970000
Generation 4, Trees 40, Best cost: 0.00482708, Est: 0.01206770, h: 3.940000
Generation 5, Trees 40, Best cost: 0.00751986, Est: 0.01879965, h: 3.910000
Generation 6, Trees 40, Best cost: 0.01064351, Est: 0.02660877, h: 3.880000
Generation 7, Trees 40, Best cost: 0.00977092, Est: 0.02442729, h: 3.880000
Generation 8, Trees 40, Best cost: 0.01755431, Est: 0.04388577, h: 3.850000
Generation 9, Trees 40, Best cost: 0.01266806, Est: 0.03167016, h: 3.850000
[0.01266806 0.01613898 0.0162402  0.0170702  0.01755431 0.01830413
 0.02128585 0.02188972 0.02657955]
[[0.05277536]
 [0.01487751]
 [0.00756818]
 [0.00527892]
 [0.00482708]
 [0.00751986]
 [0.01064351]
 [0.00977092]
 [0.01755431]
 [0.01266806

In [20]:
import pack_ga2
importlib.reload(pack_ga2)

kgs.debugging_mode = 2

runner = pack_ga2.Orchestrator(n_generations=10)
runner.ga = pack_ga2.GASinglePopulationOld(N_trees_to_do=40)

runner.ga.population_size = 100
runner.ga.selection_size = [1,2,3,4,5,10,20,30,50]
runner.ga.fixed_h = 4.
runner.ga.reduce_h_threshold = 0.01
runner.ga.reduce_h_amount = 0.03

runner.run()
print(runner.ga.population.fitness)
print(runner.ga.best_costs_per_generation)


/mnt/d/packing/code/analysis/../core/pack_dynamics.py:61: VisibleDeprecationWarning: This function is deprecated and will be removed in a future release. Use the cupy.from_dlpack() array constructor instead.
  x0 = from_dlpack(x0.toDlpack())
/mnt/d/packing/code/analysis/../core/pack_dynamics.py:93: VisibleDeprecationWarning: This function is deprecated and will be removed in a future release. Use the cupy.from_dlpack() array constructor instead.
  return from_dlpack(tmp_cost[:N].toDlpack()), from_dlpack(res.toDlpack())


Generation 0: Best costs = [0.052775364369153976]
Generation 1: Best costs = [0.014877505600452423]
Generation 2: Best costs = [0.015357094816863537]
Generation 3: Best costs = [0.012289338745176792]
Generation 4: Best costs = [0.012848918326199055]
Generation 5: Best costs = [0.011257318779826164]
Generation 6: Best costs = [0.017810005694627762]
Generation 7: Best costs = [0.015877146273851395]
Generation 8: Best costs = [0.014126379042863846]
Generation 9: Best costs = [0.012321148067712784]
[0.01232115 0.01234639 0.01244105 0.01379287 0.01412638 0.01477118
 0.01642179 0.0207346  0.03511301]
[[np.float32(0.052775364), np.float32(0.014877506), np.float32(0.015357095), np.float32(0.012289339), np.float32(0.012848918), np.float32(0.011257319), np.float32(0.017810006), np.float32(0.015877146), np.float32(0.014126379), np.float32(0.012321148)]]
